# NOTE:

This is a __single-process and multithreaded__ version of the code, which is subject to Python's GIL (global interpreter lock). When processing many datasets, prefer using the multiprocess version in `analysis-model.py`.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
from pathlib import Path
import os

Definitions:
- $W_i$ is the number of votes for the winning party in unit $i$.
- $N_i$ is the number of registered voters in unit $i$.
- $V_i$ is the number of valid votes in unit $i$.

# Cleaning steps
- Remove N/A rows
- Remove units where $W_i = 0$
- Remove units where $W_i > N_i$ 
- Remove units where $N_i < 100$
- Remove units in region `"OAV"`

In [2]:
def clean(df):
    df = df.dropna(thresh=2)
    df = df[df["votes_bbm"] > 0]
    df = df[df["votes_bbm"] <= df["registered_voters"]]
    df = df[df["registered_voters"] >= 100]
    df = df[df["reg_name"] != "OAV"]
    return df

In [3]:
# Load all CSVs in folder
path_data_root = Path("./datasets")
paths_csv = filter(lambda x : x.suffix == ".csv", path_data_root.iterdir())
# store all datasets in dictionary (filename -> dataframe)
dataframes = {path.stem : clean(pd.read_csv(path)) for path in paths_csv}
dataframes.keys()

dict_keys(['ZAMBALES'])

# Implementing the voter model

Definitions from the _supplementary notes_ attached to the reference paper.

- The average turnout over all units, $\bar{a}$, is given by
$\displaystyle\bar{a} = \frac{1}{n}\sum_i \frac{V_i}{N_i}$, with SD $s_a$.

- The mean fraction of votes, $\bar{v}$, is given by
$\displaystyle\bar{v} = \frac{1}{n}\sum_i \frac{W_i}{V_i}$, with SD $s_v$.

- A country is separated into $n$ electoral units (indexed by $i$), and each unit $i$ has an electorate of $N_i$ people and $V_i$ valid votes. In each unit, the number of votes for the winning candidate is $W_i$.

- The fraction of valid votes for the winning party in unit $i$ is denoted $v_i$, given by $v_i = \dfrac{W_i}{V_i}$.
- The turnover rate in unit $i$ is denoted $a_i$, given by $a_i = \dfrac{V_i}{N_i}$.

**I. Empirical distributions**

We are considering the empirical distribution function (edf) for votes for the winning party, rounded to entire percent values. In other words, for each of the intervals $(0\%, 0.5\%], (0.5\%, 1.5\%], ..., (98.5\%, 99.5\%], (99.5\%, 100\%]$, we count how many units $i$ the fraction $v_i = \dfrac{W_i}{V_i}$ falls within the interval. 

In [4]:
def compute_edf(df):
    # compute all the v_i's
    V = df["votes_bbm"] / df["total_votes"]
    
    # bin into discrete pdf
    divisions = 100 # the centers of the intervals are 0..100
    binned = pd.cut(V, bins=[x / float(divisions) - 0.005 for x in range(0,divisions+2)])
    binned_count = binned.value_counts().sort_index()
    intervals = pd.IntervalIndex(binned_count.index)
    edf_vi = pd.DataFrame(
        {"x": (intervals.left + intervals.right) / 2,
         "y": binned_count})
    
    # compute all the a_i's
    A = df["total_votes"] / df["registered_voters"] 
    
    # bin into discrete pdf
    divisions = 100 # the centers of the intervals are 0..100
    binned = pd.cut(A, bins=[x / float(divisions) - 0.005 for x in range(0,divisions+2)])
    binned_count = binned.value_counts().sort_index()
    intervals = pd.IntervalIndex(binned_count.index)
    edf_ai = pd.DataFrame(
        {"x": (intervals.left + intervals.right) / 2,
         "y": binned_count})
    
    return [edf_vi, edf_ai]

**II. Model Parameters**

In order to execute the model, we need to compute the model parameters $v$, $a$, $\sigma_v^L$, $\sigma_v^R$, $\sigma_a$.

- $v$ is the number of votes where the empirical distribution function for $v_i$ is maximum. To find $v$, we find the interval with the maximum number of vote ratio, and let $v$ be the center of this interval.

- $a$ is the turnout where the empirical distribution function for $a_i$ is maximum. To find $a$, we find the interval with the maximum turnout ratio, and let $a$ be the center of this interval.

- $\sigma_v^L$ is the left-sided mean deviation from $v$. This is computed as $\displaystyle\sigma_v^L = \sqrt{\left\langle(v_i - v)^2\right\rangle_{v_i < v}}$. The notation $\left\langle(v_i - v)^2\right\rangle_{v_i < v}$ refers to the expected value of $(v_i - v)^2$ for all $v_i < v$.

- $\sigma_v^R$ is the right-sided mean deviation from $v$. This is computed as $\displaystyle\sigma_v^R = \sqrt{\left\langle(v_i - v)^2\right\rangle_{v_i > v}}$. 

- $\sigma_a$ is computed as $\displaystyle\sqrt{\left\langle(a_i - a)^2\right\rangle_{(a_i < a) \wedge (v_i < v)}}$.

Lastly, based on the reference paper, $\sigma_x = 0.075$.

In [5]:
def compute_static_model_parameters(df):
    #edf = compute_edf(df)
    [edf_vi, edf_ai] = compute_edf(df)
    
    # vote ratio distribution
    #V_series = df["votes_for_winner_ratio"]
    V_series = df["votes_bbm"] / df["total_votes"]
    
    # return the entry in column x at the row which maximizes y
    v = edf_vi["x"][edf_vi.idxmax().y] 
    
    # turnout distribution
    #A_series = df["voter_turnout_ratio"]
    A_series = df["total_votes"] / df["registered_voters"]
    # return the entry in column x at the row which maximizes y
    a = edf_ai["x"][edf_ai.idxmax().y] 
    
    # apply the function (x - v) ** 2 to all entries in the fitered series, 
    # then get the mean of the resulting series
    sv_L = np.sqrt(np.mean(V_series[V_series < v].apply(lambda x: (x - v) ** 2)))
    sv_R = np.sqrt(np.mean(V_series[V_series > v].apply(lambda x: (x - v) ** 2)))
    sa = np.sqrt(np.mean(A_series[(A_series < a) & (V_series < v)].apply(lambda x: (x - a) ** 2)))
    return {
        "v": v,
        "a": a,
        "sv_L": sv_L,
        "sv_R": sv_R,
        "sa": sa,
        "sx": 0.075
    }

**III. Generative Model**

**1. Fraud Parameters**

Probabilities for fraud are assumed to be mutually exclusive. That is, for any given unit $i$,

- P(increment fraud) $ =f_i$,

- P(extreme fraud) $ =f_e$,

- P(no fraud) $ =1-f_i-f_e$.

A third fraud parameter $\alpha$ is used to describe how incremental fraud is carried out.

**2. Model Generation**

Given the values ``model_param = compute_static_model_parameters()``, and fraud parameters ``f_i``, ``f_e``, ``alpha``, a sample voting data ``opt_gen_model`` is generated as follows:
    
1. Get the number of voters $N_i =$ ``N`` for unit $i$.
    
2. Generate the number of valid votes ``a_m``, drawn from $\mathcal{N}\left(a,[\sigma_a]^2\right)$.

3. Generate the number of votes for the winner ``v_m``, drawn from $\mathcal{N} \left( v, \left[\sqrt{(2)}\sigma_v^L\right]^2 \right)$.

4. Generate fraud intensity ``x``, drawn from a folded normal distribution $\left| \mathcal{N} \left(0, \left[\sqrt{\sigma_v^R}\right]^2 \right) \right|$. If ``x`` $\notin(0,1)$, set ``x``$=0$.

    With probability $f_i$, ``v_m`` takes on the value $N\left[ a_mv_m + x\left(1-a_m\right) + x^\alpha\left(1-v_m\right)a_m \right]$. 
    
5. Generate fraud intensity ``y``, drawn from a folded normal distribution $1-\left| \mathcal{N} \left(1, \left[\sigma_x\right]^2 \right) \right|$. If ``y`` $\notin(0,1)$, set ``y``$=0$.

    With probability $f_e$, ``v_m`` takes on the value $N\left[ a_mv_m + y\left(1-a_m\right) + y^\alpha\left(1-v_m\right)a_m \right]$. 

In [6]:
def opt_gen_model(df, f_i, f_e, alpha, model_param):
    # step i (vectorized), pick electorates from the data
    N = df["registered_voters"]
    n = len(N)
    
    # step ii and iii (vectorized), compute model turnout and valid votes for winner
    a_m = scipy.stats.norm.rvs(loc=model_param["a"], scale=model_param["sa"], size=n)
    v_m = scipy.stats.norm.rvs(loc=model_param["v"], scale=np.sqrt(2)*model_param["sv_L"], size=n)
    # compute new total votes: total/winner vote should not exceed Ni, so cap them at Ni
    new_total_votes = np.minimum(a_m * N, N)
    
    # step iv (vectorized), compute results if incremental fraud occurs
        # generate x_i from folded normal
        # if x[i] not in (0,1) set to 0
    x = scipy.stats.foldnorm.rvs(loc=0, c=np.sqrt(model_param["sv_R"]), size=n)
    x = np.where((x >= 1) | (x <= 0), 0, x)
    incremental_fraud_results = N*( (a_m*v_m) + (x*(1-a_m)) + ((x**alpha)*(1-v_m)*a_m) )
    
    # step v (vectorized), compute results if extreme fraud occurs
        # generate y_i from folded normal
        # if y[i] not in (0,1) set to 0
    y = 1 - scipy.stats.foldnorm.rvs(loc=1, c=model_param["sx"], size=n)
    y = np.where((y >= 1) | (y <= 0), 0, y)
    extreme_fraud_results = N*( (a_m*v_m) + (y*(1-a_m)) + ((y**alpha)*(1-v_m)*a_m) )
    
    # selective assignment to each row of new_votes_for_winner depending on random seed
    rseed = scipy.stats.uniform.rvs(size=n) # generate n-vector from U(0,1), use scipy for consistency
    
    # cases for seed
    # 0 <= rseed < f_i:         assume incremental fraud occurs
    # f_i <= rseed < f_i + f_e: assume extreme fraud occurs
    # f_i + f_e <= rseed <= 1:  assume no fraud
    new_votes_for_winner = np.select(
        [rseed < f_i,
         rseed < f_i + f_e,
         rseed >= f_i + f_e],
        [
            incremental_fraud_results,
            extreme_fraud_results,
            N *a_m * v_m
        ]
    )
    # winner vote should not exceed total and be below 0, so bound them at [0, total]
    new_votes_for_winner = np.clip(new_votes_for_winner, 0, new_total_votes)
    
    return new_votes_for_winner / new_total_votes

`opt_compute_param_table` 

Computes **one iteration** of the parameter optimization. The implementation uses multithreading; each evaluation of $S(f_i, f_e, \alpha)$ is split across multiple threads.

Runs in around 30 mins CPU time / 16 mins Wall time on an Intel Core i7-11700 CPU.

Usage: `opt_compute_param_table(df, "grid_index.csv")`

Note: `grid_index` contains values for $f_i$, $f_e$, $\alpha$ to avoid `for` loops

In [7]:
import threading
import concurrent.futures

In [8]:
def loop_iter_thr(df, pdf_vi, model_param, fi, fe, alp,idx):
    return (((pdf_vi - opt_gen_model(df,fi,fe,alp, model_param)) / pdf_vi)**2).sum()

def opt_compute_param_table(df, grid_index_csv):
    global loop_results_S
    param_space = pd.read_csv("grid_index.csv")
    param_space_n = len(param_space)
    
    # pdf_vi is constant for all iterations
    pdf_vi = df["votes_bbm"] / df["total_votes"]
    
    # model_param is constant for all iterations
    model_param = compute_static_model_parameters(df)

    with concurrent.futures.ThreadPoolExecutor(max_workers=None) as executor:
        loop_results_S = [x for x in executor.map(loop_iter_thr,
                                    *(zip(*((df, pdf_vi, model_param, data["f_i"], data["f_e"], data["alpha"], idx) for idx, data in param_space.iterrows()))))]
        
    param_space["S"] = loop_results_S
        
    return param_space

In [9]:
%%time
test_results = opt_compute_param_table(dataframes["ZAMBALES"], "grid_index.csv")


CPU times: total: 12min 4s
Wall time: 11min 23s


Computes and saves the parameter table for 100 iterations on all files.

In [ ]:
%%time
Path("./results-model").mkdir(exist_ok=True)
for filename, df in dataframes.items():
    print(f"[[{filename}]]")
    folder_path = Path(f"./results-model/{filename}-Scomputations")
    if not folder_path.exists():
        os.makedirs(folder_path)
    for iteration in range(1,101):
        output_path = Path(f"./results-model/{filename}-Scomputations/iteration{iteration}.csv")
        if output_path.exists():
            if iteration %25 == 0:
                print(f"skip it{iteration}")
            continue
        param_table = opt_compute_param_table(df, "grid_index.csv")
        param_table.to_csv(output_path)
        print(f"end compute it{iteration}")

[[ZAMBALES]]
end compute it1
end compute it2
end compute it3
end compute it4
end compute it5
end compute it6
end compute it7
end compute it8
end compute it9
end compute it10
end compute it11
end compute it12
end compute it13
end compute it14
end compute it15
end compute it16
end compute it17
end compute it18
end compute it19
end compute it20
end compute it21
end compute it22
end compute it23
end compute it24
end compute it25
end compute it26
end compute it27
end compute it28
end compute it29
end compute it30
end compute it31
end compute it32
end compute it33
end compute it34
end compute it35
end compute it36
end compute it37
end compute it38
end compute it39
end compute it40
end compute it41
end compute it42
end compute it43
end compute it44
end compute it45
end compute it46
end compute it47
end compute it48
end compute it49
end compute it50
end compute it51
end compute it52
end compute it53
end compute it54
end compute it55
end compute it56
end compute it57
end compute it58
end comput